In [ ]:
import sqlite3
import pandas as pd
import re
from bs4 import BeautifulSoup

In [2]:
cx = sqlite3.connect("./data/database.sqlite")
cs = cx.cursor()

countries = pd.read_sql_query("SELECT * FROM COUNTRY", con=cx)
matches = pd.read_sql_query("SELECT * FROM MATCH", con=cx)
players = pd.read_sql_query("SELECT * FROM PLAYER", con=cx)
teams = pd.read_sql_query("SELECT * FROM TEAM", con=cx)
leagues = pd.read_sql_query("SELECT * FROM LEAGUE", con=cx)
transfers =pd.read_csv("./data/transfers.csv")

query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, con=cx)

transfers.head()

,Name,Position,Age,Team_from,League_from,Team_to,League_to,Season,Market_value,Transfer_fee
0,Luís Figo,Right Winger,27,FC Barcelona,LaLiga,Real Madrid,LaLiga,2000-2001,NaN,60000000
1,Hernán Crespo,Centre-Forward,25,Parma,Serie A,Lazio,Serie A,2000-2001,NaN,56810000
2,Marc Overmars,Left Winger,27,Arsenal,Premier League,FC Barcelona,LaLiga,2000-2001,NaN,40000000
3,Gabriel Batistuta,Centre-Forward,31,Fiorentina,Serie A,AS Roma,Serie A,2000-2001,NaN,36150000
4,Nicolas Anelka,Centre-Forward,21,Real Madrid,LaLiga,Paris SG,Ligue 1,2000-2001,NaN,34500000


In [3]:
clean_transfers = transfers.drop(columns=["Market_value"])
clean_transfers.head()

,Name,Position,Age,Team_from,League_from,Team_to,League_to,Season,Transfer_fee
0,Luís Figo,Right Winger,27,FC Barcelona,LaLiga,Real Madrid,LaLiga,2000-2001,60000000
1,Hernán Crespo,Centre-Forward,25,Parma,Serie A,Lazio,Serie A,2000-2001,56810000
2,Marc Overmars,Left Winger,27,Arsenal,Premier League,FC Barcelona,LaLiga,2000-2001,40000000
3,Gabriel Batistuta,Centre-Forward,31,Fiorentina,Serie A,AS Roma,Serie A,2000-2001,36150000
4,Nicolas Anelka,Centre-Forward,21,Real Madrid,LaLiga,Paris SG,Ligue 1,2000-2001,34500000


In [4]:
print("Tables in database:")
print(tables)

Tables in database:
                name
0    sqlite_sequence
1  Player_Attributes
2             Player
3              Match
4             League
5            Country
6               Team
7    Team_Attributes


In [5]:
print("columns of matches table:", " | ".join(matches.columns))
matches.head()


columns of matches table: id | country_id | league_id | season | stage | date | match_api_id | home_team_api_id | away_team_api_id | home_team_goal | away_team_goal | home_player_X1 | home_player_X2 | home_player_X3 | home_player_X4 | home_player_X5 | home_player_X6 | home_player_X7 | home_player_X8 | home_player_X9 | home_player_X10 | home_player_X11 | away_player_X1 | away_player_X2 | away_player_X3 | away_player_X4 | away_player_X5 | away_player_X6 | away_player_X7 | away_player_X8 | away_player_X9 | away_player_X10 | away_player_X11 | home_player_Y1 | home_player_Y2 | home_player_Y3 | home_player_Y4 | home_player_Y5 | home_player_Y6 | home_player_Y7 | home_player_Y8 | home_player_Y9 | home_player_Y10 | home_player_Y11 | away_player_Y1 | away_player_Y2 | away_player_Y3 | away_player_Y4 | away_player_Y5 | away_player_Y6 | away_player_Y7 | away_player_Y8 | away_player_Y9 | away_player_Y10 | away_player_Y11 | home_player_1 | home_player_2 | home_player_3 | home_player_4 | home_player_5

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [6]:
clean_matches = matches[["country_id", "league_id", "season", "home_team_api_id", "away_team_api_id", "home_team_goal", "away_team_goal", "goal", "shoton", "shotoff", "possession"]]

clean_matches.head()

,country_id,league_id,season,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,goal,shoton,shotoff,possession
0,1,1,2008/2009,9987,9993,1,1,None,None,None,None
1,1,1,2008/2009,10000,9994,0,0,None,None,None,None
2,1,1,2008/2009,9984,8635,0,3,None,None,None,None
3,1,1,2008/2009,9991,9998,5,0,None,None,None,None
4,1,1,2008/2009,7947,9985,1,3,None,None,None,None


In [7]:
clean_matches = pd.merge(clean_matches, countries[["id", "name"]], left_on="country_id", right_on="id", how="left")
clean_matches = clean_matches.drop(columns=["id", "country_id"])
clean_matches = clean_matches.rename(columns={"name": "country"})

clean_matches = pd.merge(clean_matches, leagues[["id", "name"]], left_on="league_id", right_on="id", how="left")
clean_matches = clean_matches.drop(columns=["id", "league_id"])
clean_matches = clean_matches.rename(columns={"name": "league"})

clean_matches = pd.merge(clean_matches, teams[["team_api_id", "team_long_name"]], left_on="home_team_api_id", right_on="team_api_id", how="left")
clean_matches = clean_matches.drop(columns=["team_api_id", "home_team_api_id"])
clean_matches = clean_matches.rename(columns={"team_long_name": "home_team"})

clean_matches = pd.merge(clean_matches, teams[["team_api_id", "team_long_name"]], left_on="away_team_api_id", right_on="team_api_id", how="left")
clean_matches = clean_matches.drop(columns=["team_api_id", "away_team_api_id"])
clean_matches = clean_matches.rename(columns={"team_long_name": "away_team"})

clean_matches = clean_matches[["season", "country", "league", "home_team", "away_team", "home_team_goal", "away_team_goal", "goal", "shoton", "shotoff", "possession"]]

clean_matches['season'] = clean_matches['season'].str.replace('/', '-')

clean_matches.head()

,season,country,league,home_team,away_team,home_team_goal,away_team_goal,goal,shoton,shotoff,possession
0,2008-2009,Belgium,Belgium Jupiler League,KRC Genk,Beerschot AC,1,1,None,None,None,None
1,2008-2009,Belgium,Belgium Jupiler League,SV Zulte-Waregem,Sporting Lokeren,0,0,None,None,None,None
2,2008-2009,Belgium,Belgium Jupiler League,KSV Cercle Brugge,RSC Anderlecht,0,3,None,None,None,None
3,2008-2009,Belgium,Belgium Jupiler League,KAA Gent,RAEC Mons,5,0,None,None,None,None
4,2008-2009,Belgium,Belgium Jupiler League,FCV Dender EH,Standard de Liège,1,3,None,None,None,None


In [8]:
def extract_final_possession(possession_xml):
    """Extract the final home and away possession values from XML string."""
    if pd.isna(possession_xml) or not isinstance(possession_xml, str):
        return None, None
    
    try:
        soup = BeautifulSoup(f"<root>{possession_xml}</root>", "xml")        
        values = soup.find_all('value')
        if not values: 
            return None, None
        
        last_value = values[-1]        
        home_pos = float(last_value.find('homepos').text) if last_value.find('homepos') else None
        away_pos = float(last_value.find('awaypos').text) if last_value.find('awaypos') else None
        
        return home_pos, away_pos
    
    except Exception as e:
        try:
            home_matches = re.findall(r'<homepos>(\d+\.?\d*)</homepos>', possession_xml)
            away_matches = re.findall(r'<awaypos>(\d+\.?\d*)</awaypos>', possession_xml)
            if home_matches and away_matches:
                return float(home_matches[-1]), float(away_matches[-1])
            return None, None
        except Exception as fallback_error:
            return None, None

possession_results = clean_matches['possession'].apply(extract_final_possession)
possession_df = pd.DataFrame(
    possession_results.tolist(), 
    columns=['home_possession', 'away_possession'],
    index=clean_matches.index
)

clean_matches = pd.concat([clean_matches, possession_df], axis=1)
clean_matches.drop(columns=["possession"], inplace=True)
clean_matches.head()

,season,country,league,home_team,away_team,home_team_goal,away_team_goal,goal,shoton,shotoff,home_possession,away_possession
0,2008-2009,Belgium,Belgium Jupiler League,KRC Genk,Beerschot AC,1,1,None,None,None,NaN,NaN
1,2008-2009,Belgium,Belgium Jupiler League,SV Zulte-Waregem,Sporting Lokeren,0,0,None,None,None,NaN,NaN
2,2008-2009,Belgium,Belgium Jupiler League,KSV Cercle Brugge,RSC Anderlecht,0,3,None,None,None,NaN,NaN
3,2008-2009,Belgium,Belgium Jupiler League,KAA Gent,RAEC Mons,5,0,None,None,None,NaN,NaN
4,2008-2009,Belgium,Belgium Jupiler League,FCV Dender EH,Standard de Liège,1,3,None,None,None,NaN,NaN


In [9]:
clean_matches['home_win'] = (clean_matches['home_team_goal'] > clean_matches['away_team_goal']).astype(int)
clean_matches['home_loss'] = (clean_matches['home_team_goal'] < clean_matches['away_team_goal']).astype(int)
clean_matches['home_tie'] = (clean_matches['home_team_goal'] == clean_matches['away_team_goal']).astype(int)

clean_matches['away_win'] = (clean_matches['home_team_goal'] < clean_matches['away_team_goal']).astype(int)
clean_matches['away_loss'] = (clean_matches['home_team_goal'] > clean_matches['away_team_goal']).astype(int)
clean_matches['away_tie'] = (clean_matches['home_team_goal'] == clean_matches['away_team_goal']).astype(int)

clean_matches['match_count'] = 1

home_stats = clean_matches.groupby(['season', 'home_team', 'country', 'league'])[['home_team_goal', 'away_team_goal', 'home_win', 'home_loss', 'home_tie', 'match_count']].sum().reset_index()
home_stats = home_stats.rename(columns={'home_team': 'team',
                                       'home_team_goal': 'goals_scored_home', 
                                       'away_team_goal': 'goals_conceded_home', 
                                       'home_win': 'wins_home', 
                                       'home_loss': 'losses_home', 
                                       'home_tie': 'ties_home',
                                       'match_count': 'home_matches'})

away_stats = clean_matches.groupby(['season', 'away_team'])[['home_team_goal', 'away_team_goal', 'away_win', 'away_loss', 'away_tie', 'match_count']].sum().reset_index()
away_stats = away_stats.rename(columns={'away_team': 'team',
                                       'home_team_goal': 'goals_conceded_away', 
                                       'away_team_goal': 'goals_scored_away', 
                                       'away_win': 'wins_away', 
                                       'away_loss': 'losses_away', 
                                       'away_tie': 'ties_away',
                                       'match_count': 'away_matches'})

team_stats = pd.merge(home_stats, away_stats, on=['season', 'team'], how='outer')
team_stats['total_matches'] = team_stats['home_matches'] + team_stats['away_matches']
team_stats = team_stats.fillna(0)


In [10]:
team_stats.head()

,season,team,country,league,goals_scored_home,goals_conceded_home,wins_home,losses_home,ties_home,home_matches,goals_conceded_away,goals_scored_away,wins_away,losses_away,ties_away,away_matches,total_matches
0,2008-2009,1. FC Köln,Germany,Germany 1. Bundesliga,14,25,4,8,5,17,25,21,7,9,1,17,34
1,2008-2009,AC Bellinzona,Switzerland,Switzerland Super League,29,23,8,6,4,18,28,15,3,9,6,18,36
2,2008-2009,ADO Den Haag,Netherlands,Netherlands Eredivisie,21,18,5,7,5,17,40,20,3,11,3,17,34
3,2008-2009,AJ Auxerre,France,France Ligue 1,18,12,8,5,6,19,23,17,8,10,1,19,38
4,2008-2009,AS Monaco,France,France Ligue 1,26,23,7,8,4,19,22,15,4,7,8,19,38


In [ ]:
home_possession_stats = clean_matches.groupby(['season', 'home_team', 'country', 'league'])['home_possession'].mean().reset_index()
home_possession_stats = home_possession_stats.rename(columns={
    'home_team': 'team',
    'home_possession': 'avg_possession_home'
})
away_possession_stats = clean_matches.groupby(['season', 'away_team'])['away_possession'].mean().reset_index()
away_possession_stats = away_possession_stats.rename(columns={
    'away_team': 'team',
    'away_possession': 'avg_possession_away'
})
team_stats = pd.merge(team_stats, home_possession_stats[['season', 'team', 'avg_possession_home']], 
                      on=['season', 'team'], how='left')
team_stats = pd.merge(team_stats, away_possession_stats[['season', 'team', 'avg_possession_away']], 
                      on=['season', 'team'], how='left')
team_stats['avg_possession_overall'] = (
    (team_stats['avg_possession_home'] * team_stats['home_matches'] + 
     team_stats['avg_possession_away'] * team_stats['away_matches']) / 
    team_stats['total_matches']
)
team_stats.head()

,season,team,country,league,goals_scored_home,goals_conceded_home,wins_home,losses_home,ties_home,home_matches,goals_conceded_away,goals_scored_away,wins_away,losses_away,ties_away,away_matches,total_matches,avg_possession_home,avg_possession_away,avg_possession_overall
0,2008-2009,1. FC Köln,Germany,Germany 1. Bundesliga,14,25,4,8,5,17,25,21,7,9,1,17,34,52.25,40.2,46.225
1,2008-2009,AC Bellinzona,Switzerland,Switzerland Super League,29,23,8,6,4,18,28,15,3,9,6,18,36,NaN,NaN,NaN
2,2008-2009,ADO Den Haag,Netherlands,Netherlands Eredivisie,21,18,5,7,5,17,40,20,3,11,3,17,34,NaN,NaN,NaN
3,2008-2009,AJ Auxerre,France,France Ligue 1,18,12,8,5,6,19,23,17,8,10,1,19,38,NaN,NaN,NaN
4,2008-2009,AS Monaco,France,France Ligue 1,26,23,7,8,4,19,22,15,4,7,8,19,38,NaN,NaN,NaN


In [ ]:
# team_stats.to_csv("./data/team_stats.csv", index=False)
# clean_transfers.to_csv("./data/clean_transfers.csv", index=False)

team_stats.to_json("./data/team_stats.json", orient="records")
clean_transfers.to_json("./data/clean_transfers.json", orient="records")